In [ ]:
import pandas as pd
import scipy
import scipy.spatial
import networkx as nx
import numpy as np
from collections import deque
import collections
import pickle

# for plotting graphs
import matplotlib.pyplot as plt

from CommonDefs import Point, Edge 
from TwoDimConvexHull import TwoDimConvexHull, PrintTwoDimConvexHull
from Utils import nearestKNeighboursOfEdgeMidPt, checkTriangleForDelaunayCriteria
from Graph import Graph
from DTEnv import DTEnv
from DTDoubleDQNAgent import DTDoubleDQNAgent

In [ ]:
pd.options.display.max_columns = None

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv("D:\Thesis_Experiments\Data\DT_5_sorted.txt", sep=" ", header=None)
df.head()


In [ ]:
df.shape

In [ ]:
pointData = {}
optimalDT = {}
for i in range(len(df)):
    pointId = 1
    points = []
    for j in range(0 , len(df.columns), 2):
        if df.loc[i, j] == "output":
            triList = []
            for k in range(j+1, len(df.columns), 3):
                tri=[df.loc[i, k], df.loc[i, k+1], df.loc[i, k+2]]
                tri = sorted(tri)
                triList.append(tri)
            optimalDT[i] = triList
            break
        else:
            points.append(Point(pointId, df.loc[i, j], df.loc[i, j + 1]))
            pointId = pointId + 1
    pointData[i] = points

In [ ]:
# Initialising the environment
env = DTEnv(2)
agent = DTDoubleDQNAgent(env)

In [ ]:
initializeModel = True

totalPredictedTriangles = 0
totalNumTrisInGroundTruth = 0
trianglesCorrectlyPredicted = 0
validDelaunayTriangles = 0
ti = 0

for i in range(len(df)):     

    score = 0
    env.reset()
    env.initialize(pointData[i])  

    if initializeModel:
        agent.initializeModelForValidation(env, 'DDQN_Delaunay.h5')
        initializeModel = False

    _, state = env.getStartState()
    terminal_state = False

    while not terminal_state:

        #Get the free edge from the list
        edgeToProcess = env.getEdgesToProcess()[0]

        action = agent.get_action_for_validation(state, edgeToProcess)            

        '''if not action:
            break'''

        #print('edgeToProcess: {}-{}'.format(edgeToProcess[0], edgeToProcess[1]))
        #print('action chosen: {}'.format(env.getNodeIdFromPosAttr( action[4], action[5])))

        reward = env.getReward(edgeToProcess, action)        
        next_state = env.getNextState(edgeToProcess, action)
        env.removeProcessedEdge(edgeToProcess)
        terminal_state = env.isTerminalState()

        score += reward
        state = next_state

    if terminal_state:
        print('generated triangles: {}'.format(env.getGeneratedTriangles()))
        
        trianglesPredicted = env.getGeneratedTriangles()
        trianglesGroundTruth = optimalDT[i]
        
        totalPredictedTriangles += len(trianglesPredicted)
        totalNumTrisInGroundTruth += len(trianglesGroundTruth)
        
        if len(trianglesGroundTruth) == len(trianglesPredicted):
            ti += 1
        
        for tri in trianglesPredicted:
            if tri in trianglesGroundTruth:
                trianglesCorrectlyPredicted += 1
            pt1 = env.getPointFromId(tri[0])
            pt2 = env.getPointFromId(tri[1])
            pt3 = env.getPointFromId(tri[2])
            if (checkTriangleForDelaunayCriteria(pt1, pt2, pt3, pointData[i]) == True):
                validDelaunayTriangles += 1

print('TCA: {}'.format((ti/len(df)) * 100))
print('Total Number of ground truth triangles: {}'.format(totalNumTrisInGroundTruth))
print('Total Number of triangles predicted: {}'.format(totalPredictedTriangles))
print('Number of triangles correctly predicted: {}'.format(trianglesCorrectlyPredicted))
print('Number of valid delaunay triangles: {}'.format(validDelaunayTriangles))

print('Triangle Coverage: {}'.format( (trianglesCorrectlyPredicted/totalPredictedTriangles) * 100))
print('DT Rate: {}'.format( (validDelaunayTriangles/totalPredictedTriangles) * 100))
      


      

    
    